# Statistics for Missing Data

## Train Data (No NA in both columns):

### Essential Column - 44 NA
### Chromosome - 1 NA

## Total Incomplete records -> 45 -> 45/863 -> 5.21%



## Test Data (2 records with NA in both Columns):

### Essential Column - 17 NA
### Chromosome - 59 NA

## Total Incomplete records -> 74 -> 74/382 -> 19.37%

# Methods for dealing with NA's

## MissForest, uses a random forest to predict values for the NA values, going column by column

# NOTE NEED TO DISCUSS THAT WE NEED TO USE BOTH TRAIN AND TEST DATA AT SAME TIME WHEN REPLACING NA's SINCE SAME METHOD SHOULD BE USED FOR BOTH

In [81]:
# Install a pip package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install missingpy

import os

import re

import pandas as pd

from IPython.display import display

from missingpy import MissForest



os.chdir(r'D:\Documents\WATERLOO\3B\STAT 441\protein-localization')

# Get Column labels from Description file

description = pd.read_csv (r'D:\Documents\WATERLOO\3B\STAT 441\protein-localization\field_descriptions.txt', 
                           sep='\t', lineterminator='\n', header = None)

col_labels = description[0]

def clean_labels(label):
    label = re.sub(' +', ' ',label)
    label = re.sub(':', '',label)
    return "".join(label.rstrip().lstrip())

col_labels = col_labels.apply(clean_labels)



# Import Data
train = pd.read_csv (r'D:\Documents\WATERLOO\3B\STAT 441\protein-localization\train.csv', header=None,names = col_labels,na_values='?')

test = pd.read_csv (r'D:\Documents\WATERLOO\3B\STAT 441\protein-localization\test.csv', header=None,names = col_labels,na_values='?')


# Find Column numbers for the Interaction Columns
begin_interact = train.columns.get_loc("INTERACTING PROTEIN P239476 Type") 
end_interact = train.columns.get_loc("INTERACTING PROTEIN P235069 Corr")


# Generate seperate data sets for different situations\

# Without Interaction Columns
TEST_Data_1 = test.drop(test.iloc[:, begin_interact:end_interact+1], inplace = False, axis = 1)

TRAIN_Data_1 = train.drop(train.iloc[:, begin_interact:end_interact+1], inplace = False, axis = 1)


# Without Interaction Columns, last 15 solution columns
TEST_Data_2 =  TEST_Data_1.iloc[:, :-16]

TRAIN_Data_2 =  TRAIN_Data_1.iloc[:, :-16]


# Without Interaction Columns, last 15 solution columns,442 binary columns
TEST_Data_3 = TEST_Data_2.drop(
    TEST_Data_2.iloc[:, 2:444], inplace = False, axis = 1)

TRAIN_Data_3 = TRAIN_Data_2.drop(
    TRAIN_Data_2.iloc[:, 2:444], inplace = False, axis = 1)


In [82]:
# Encoding for dealing with Essential column
essential_encode = {"ESSENTIAL":     {"Essential": 1, "Non-Essential": 2,"Ambiguous-Essential": 3}}
reverse_essential_encode = {"ESSENTIAL":     {1: "Essential", 2: "Non-Essential",3: "Ambiguous-Essential"}}

In [83]:
#Create a data frame relabelling the categorical variables
X = TRAIN_Data_3.drop('Protein', axis=1)
X = X.replace(essential_encode)
X["ESSENTIAL"] = X.ESSENTIAL.astype('category')
X["Chromosome"] = X.Chromosome.astype('category')

#Find index of the categorical variables
cat_cols = [X.columns.get_loc(col) for col in X.select_dtypes(['category']).columns.tolist()]

#Set random_state of MissForest
imputer = MissForest(random_state=123)

#Use MissForest and the categorical variables to fill in NA values
X_imputed = imputer.fit_transform(X,cat_vars=cat_cols)

Iteration: 0
Iteration: 1
Iteration: 2


In [86]:
Complete_train= TRAIN_Data_1.copy()
Complete_train["ESSENTIAL"] = X_imputed[:,0].astype(int)
Complete_train = Complete_train.replace(reverse_essential_encode)
Complete_train["Chromosome"] =X_imputed[:,1].astype(int)

In [89]:
display(TRAIN_Data_1.isna().sum().sum())
display(Complete_train.isna().sum().sum())

45

0

In [80]:
Complete_train.to_csv(r'D:\Documents\WATERLOO\3B\STAT 441\protein-localization\TESTING_REPLACE.csv', index = False)